In [2]:
# Imports
import pandas as pd
import numpy as np
import os

In [ ]:
folder_path = r'C:\Users\bayli\Documents\Git Projects\CS2\xlsx_exports'
sheet_data = {}
all_first_kills = pd.DataFrame()

for file_name in os.listdir(folder_path):
    if file_name.endswith('.xlsx'):
        file_path = os.path.join(folder_path, file_name)

        excel_data = pd.read_excel(file_path, sheet_name=None)

        for sheet_name, df in excel_data.items():
            if 'Kills' in df:
                df_Kills = df
                first_kills = df_Kills.sort_values(by=['round_number', 'tick'])
                first_kills = first_kills.groupby('round_number').first().reset_index()
                all_first_kills = pd.concat([all_first_kills, first_kills], ignore_index=True)
            if sheet_name in sheet_data:
                sheet_data[sheet_name] = pd.concat([sheet_data[sheet_name], df], ignore_index=True)
            else:
                sheet_data[sheet_name] = df

In [4]:
df_General = sheet_data['General']
df_Rounds = sheet_data['Rounds']
df_Players = sheet_data['Players']
df_Kills = sheet_data['Kills']
df_Weapons = sheet_data['Weapons']
df_PFM = sheet_data['Players Flashbang matrix']
df_Clutches = sheet_data['Clutches']

df_Players = df_Players.drop('color', axis=1)

In [5]:
team_rounds_won = df_Rounds.groupby('winner_name').agg(
    total_rounds_won=('winner_name', 'size'),
    t_rounds_won=('winner_side', lambda x: (x == 2).sum()),
    ct_rounds_won=('winner_side', lambda x: (x == 3).sum())
).reset_index()

team_rounds_won.columns = ['team_name', 'total_rounds_won','t_rounds_won', 'ct_rounds_won']

In [6]:
player_kills = df_Kills.groupby('killer_steam_id').agg(
    kills=('killer_steam_id', 'size'),
    headshots=('is_headshot', 'sum'),
    wallbang_kills=('penetrated_objects', 'sum'),
    assisted_flashes=('is_assisted_flash', 'sum'),
    trade_kills=('is_trade_kill', 'sum'),
    trade_deaths=('is_trade_death', 'sum'),
    no_scope=('is_no_scope', 'sum'),
    through_smoke=('is_through_smoke', 'sum'),
    airbone_kills=('is_killer_airbone', 'sum'),
    airbone_victim_kills=('is_victim_airbone', 'sum'),
    blind_kills=('is_killer_blinded', 'sum'),
    victim_blind_kills=('is_victim_blinded', 'sum'),
).reset_index()


In [7]:
player_kills.rename(columns={'killer_steam_id': 'steam_id'}, inplace=True)

In [ ]:
player_kills

In [9]:
df_Players_1 = df_Players.groupby('steam_id', as_index=False).agg({'name': 'first',
    'team_name': 'first',
    'kill_count': 'sum', 
    'assist_count': 'sum',
    'kd':'mean',
    'mvp':'sum',
    'HLTV':'mean',
    'HLTV 2.0':'mean',
    'kast':'mean',
    'death_count': 'sum',
    'headshot_count': 'sum',
    'first_kill_count': 'sum',
    'first_death_count': 'sum',
    'bomb_defused_count': 'sum',
    'bomb_planted_count': 'sum',
    '1v1': 'sum',
    '1v2': 'sum',
    '1v3': 'sum',
    '1v4': 'sum',
    '1v5': 'sum',
    '1v1_won': 'sum',
    '1v2_won': 'sum',
    '1v3_won': 'sum',
    '1v4_won': 'sum',
    '1v5_won': 'sum',
    '1v2_lost': 'sum',
    '1v3_lost': 'sum',
    '1v4_lost': 'sum',
    '1v5_lost': 'sum',
    })

In [10]:
df_Players_1 = df_Players_1.merge(team_rounds_won, on='team_name', how='left')
df_Players_1 = df_Players_1.merge(player_kills, on='steam_id', how='left')

In [11]:
df_Players_1.to_csv('players.csv')